In [1]:
from llama_cpp import Llama
import textwrap
import os


In [2]:
# === SETUP ===
MODEL_PATH = "./models/mistral_lite/Mistral-7B-Instruct-v0.3-IQ4_XS.gguf"  # Adjust to your GGUF file
CHUNK_SIZE = 1200  # tokens
OVERLAP = 200      # tokens for overlap between chunks

# === LOAD LLM ===
llm = Llama(
    model_path=MODEL_PATH,
    n_gpu_layers=32,
    n_ctx=2048,
    n_threads=8,
    use_mmap=True,
    use_mlock=True,
    verbose=True
)

llama_model_loader: loaded meta data with 29 key-value pairs and 291 tensors from ./models/mistral_lite/Mistral-7B-Instruct-v0.3-IQ4_XS.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Mistral-7B-Instruct-v0.3
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:       

In [3]:
# === TOKENIZER-LIKE CHUNKING ===
def chunk_text(text, chunk_size=CHUNK_SIZE, overlap=OVERLAP):
    import tiktoken  # Optional: llama-cpp-python has its own tokenizer too
    tokenizer = llm.tokenize
    tokens = tokenizer(text.encode("utf-8"))[1:]  # Remove BOS
    chunks = []

    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        if chunk:
            decoded = llm.detokenize(chunk).decode("utf-8", errors="ignore")
            chunks.append(decoded)

    return chunks

In [4]:
# === SUMMARIZE ONE CHUNK ===
def summarize_chunk(text):
    prompt = f"""### Instruction:
You are tasked with summarizing a portion of a meeting transcript.
Keep the summary detailed, reflecting key points, even if minor.
Do not omit information you think might be relevant later.
The summary should be no more than five sentences.
This is just one part of the entire conversation.

### Input:
{text}

### Response:
"""
    result = llm(prompt, max_tokens=512, stop=["###", "### Instruction", "### Input", "### Response"])
    return result["choices"][0]["text"].strip()


In [5]:
# === GENERATE FINAL SUMMARY PROMPT ===
def summarize_final_prompt(summaries):
    joined = "\n".join(summaries)
    prompt = f"""### Instruction:
You are tasked with generating a concise and coherent summary from multiple segment summaries of a meeting.
Ensure the final summary is comprehensive, relevant, and covers the entire meeting meaningfully.
The final summary should be no more than ten sentences.
Use the segment summaries below to infer key ideas and decisions made.

### Segment Summaries:
{joined}

### Final Summary:
"""
    return prompt

In [6]:
def summarize_hierarchical(chunk_summaries, group_size=3):
    grouped_summaries = [
        chunk_summaries[i:i + group_size]
        for i in range(0, len(chunk_summaries), group_size)
    ]

    intermediate_summaries = []
    for i, group in enumerate(grouped_summaries):
        joined = "\n".join(group)
        prompt = f"""### Instruction:
You are tasked with summarizing a portion of a meeting transcript.
Keep the summary detailed, reflecting key points, even if minor.
Do not omit information you think might be relevant later.
The summary should be no more than seven sentences.
Use the segment summaries below to infer key ideas and decisions made.


### Segment Summaries:
{joined}

### Group Summary:
"""
        print(f"📝 Summarizing group {i + 1}")
        response = llm(prompt, max_tokens=256)
        summary = response["choices"][0]["text"].strip()
        intermediate_summaries.append(summary)

    return summarize_final_prompt(intermediate_summaries)


In [7]:
# === MAIN SUMMARIZATION PIPELINE ===
def summarize_transcript(full_text):
    print("🔄 Chunking transcript...")
    chunks = chunk_text(full_text)
    print(f"✅ {len(chunks)} chunks created.")

    chunk_summaries = []
    for i, chunk in enumerate(chunks):
        print(f"📝 Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarize_chunk(chunk)
        chunk_summaries.append(summary)

    print("\n🧠 Generating final summary prompt...")
    final_prompt = summarize_final_prompt(chunk_summaries)

    print("📚 Running final summary inference...")
    final_response = llm(final_prompt, max_tokens=512, stop=["###", "### Instruction", "### Segment Summaries", "### Final Summary"])
    final_summary = final_response["choices"][0]["text"].strip()

    print("\n✅ Final Summary:\n")
    print(textwrap.fill(final_summary, 100))

    return final_summary

In [8]:
# # === USAGE EXAMPLE ===
# # Replace this with your transcript
# with open("./amicorpus/ES2002c/ES2002c.Mix-Headset.txt", "r", encoding="utf-8") as f:
#     full_transcript = f.read()

# final_summary = summarize_transcript(full_transcript)

In [9]:
def process_meeting_folder(folder_path):
    base_name = os.path.basename(folder_path)
    transcript_path = os.path.join(folder_path, f"{base_name}.Mix-Headset.txt")
    summary_path = os.path.join(folder_path, f"{base_name}.summary.txt")
    
    if os.path.exists(summary_path):
        print(f"[SKIP] Summary already exists: {summary_path}")
        return
    
    if not os.path.exists(transcript_path):
        print(f"[WARN] Transcript not found: {transcript_path}")
        return

    print(f"[INFO] Processing: {transcript_path}")
    
    with open(transcript_path, "r", encoding="utf-8") as f:
        transcript = f.read()
        
    final_summary = summarize_transcript(transcript)
    
    
    output_path = os.path.join(folder_path, f"{base_name}.summary.txt")
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(final_summary)
    
    print(f"[DONE] Summary saved to: {output_path}")


In [10]:
dataset_root = "./amicorpus/"  
for folder in os.listdir(dataset_root):
    folder_path = os.path.join(dataset_root, folder)
    if os.path.isdir(folder_path):
        process_meeting_folder(folder_path)


[SKIP] Summary already exists: ./amicorpus/ES2002a\ES2002a.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2002b\ES2002b.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2002c\ES2002c.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2002d\ES2002d.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2003a\ES2003a.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2003b\ES2003b.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2003c\ES2003c.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2003d\ES2003d.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2004a\ES2004a.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2004b\ES2004b.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2004c\ES2004c.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2004d\ES2004d.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2005a\ES2005a.summary.txt
[SKIP] Summary already exists: ./amicorpus/ES2005b\ES2005b.summary.txt
[SKIP]

llama_perf_context_print:        load time =  148899.90 ms
llama_perf_context_print: prompt eval time =  148898.54 ms /  1283 tokens (  116.05 ms per token,     8.62 tokens per second)
llama_perf_context_print:        eval time =   42630.65 ms /   245 runs   (  174.00 ms per token,     5.75 tokens per second)
llama_perf_context_print:       total time =  191710.12 ms /  1528 tokens
Llama.generate: 77 prefix-match hit, remaining 1206 prompt tokens to eval


📝 Summarizing chunk 2/7...


llama_perf_context_print:        load time =  148899.90 ms
llama_perf_context_print: prompt eval time =  144503.44 ms /  1206 tokens (  119.82 ms per token,     8.35 tokens per second)
llama_perf_context_print:        eval time =   26635.79 ms /   153 runs   (  174.09 ms per token,     5.74 tokens per second)
llama_perf_context_print:       total time =  171208.88 ms /  1359 tokens
Llama.generate: 77 prefix-match hit, remaining 1206 prompt tokens to eval


📝 Summarizing chunk 3/7...


...

📝 Summarizing chunk 7/7...


llama_perf_context_print:        load time =  148899.90 ms
llama_perf_context_print: prompt eval time =    2207.29 ms /    18 tokens (  122.63 ms per token,     8.15 tokens per second)
llama_perf_context_print:        eval time =    6748.15 ms /    42 runs   (  160.67 ms per token,     6.22 tokens per second)
llama_perf_context_print:       total time =    8968.39 ms /    60 tokens
Llama.generate: 11 prefix-match hit, remaining 937 prompt tokens to eval



🧠 Generating final summary prompt...
📚 Running final summary inference...


llama_perf_context_print:        load time =  148899.90 ms
llama_perf_context_print: prompt eval time =  111988.05 ms /   937 tokens (  119.52 ms per token,     8.37 tokens per second)
llama_perf_context_print:        eval time =   51503.36 ms /   301 runs   (  171.11 ms per token,     5.84 tokens per second)
llama_perf_context_print:       total time =  163682.43 ms /  1238 tokens



✅ Final Summary:

In the meeting, the design team discussed the product's design, focusing on the remote control. The
remote was agreed upon to be made of rubber, with a yellow back and black components. The menu would
be a darker yellow to improve contrast. The LCD display and speed recognition were included, but due
to budget constraints, speech recognition was proposed to be removed. To meet the production cost of
12.5 euros, additional changes included scrapping the special yellow color and possibly simplifying
the design by removing curves. The remote's cover material may be changed to plastic, and the color
option could be offered for an additional cost. The team also considered offering separate covers
for customers who prefer custom designs.  The unique curve design was maintained, but sacrifices
were made to reduce costs, such as potentially removing the spongy feel and simplifying the design.
The LCD screen was kept, despite the added cost, for its added functionality. The g